In [1]:
import numpy as np
import Sprede as sp
import MDAnalysis as mda

In [2]:
time_step = 1.0
step_skip = 1
dt_correct = np.arange(1,200,2)
dt_correct[-1] = 200


test_u = mda.Universe('input_data/example_LAMMPS.data','input_data/example_LAMMPS.dcd')
test_u_sp = sp.MDAnalysisParser(test_u,specie=['1','2'],isotopes = ['41K','35Cl'],time_step=time_step,step_skip=step_skip)
q_points = test_u_sp.calculate_q_points(q_max = 5)
test_incoh = test_u_sp.calculate_Finc_qt(q_points)

/opt/anaconda3/lib/python3.12/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [56]:
# testing incoh shape 
assert test_incoh.shape == (len(q_points),len(test_u_sp.delta_t))


# testing correct 0th q_point
assert (test_incoh[0,:].round(3) == 9.351).all()

In [6]:
# Testing correct dt
assert dt_correct.all() == test_u_sp.delta_t.all()

In [18]:
# Testing scattering length assignement
np.sum(test_u_sp.scattering_lengths) == (52*6.1 + (256-52)*1.5)

True

In [58]:
import pytest
import numpy as np
import Sprede as sp
import MDAnalysis as mda



@pytest.fixture()
def produce_incoh():
    time_step = 1.0
    step_skip = 1
    test_u = mda.Universe('input_data/example_LAMMPS.data','input_data/example_LAMMPS.dcd')
    test_u_sp = sp.MDAnalysisParser(test_u,specie=['1','2'],isotopes = ['41K','35Cl'],time_step=time_step,step_skip=step_skip)
    q_points = test_u_sp.calculate_q_points(q_max = 5)
    test_incoh = test_u_sp.calculate_Finc_qt(q_points)
    return test_u_sp,test_incoh

def test_dt_assignement(test_u_sp):
    dt_correct = np.arange(1,200,2)
    dt_correct[-1] = 200
    assert dt_correct.all() == test_u_sp.delta_t.all()


def test_scattering_length(test_u_sp):
    assert np.sum(test_u_sp.scattering_lengths) == (52*6.1 + (256-52)*1.5)

def test_incoh_shape(test_u_sp,test_incoh):
    assert test_incoh.shape == (len(q_points),len(test_u_sp.delta_t))

def test_incoh_0th_q_point(test_incoh):
    assert (test_incoh[0,:].round(3) == 9.351).all()

